## Date Handling of Input Files

> The data contains periods in CET/CEST format.
	

* we need to have start and end date in one consitent time zone
* we need to split the period into start and end date of the period 

"01/01/2015 00:00:00 - 01/01/2015 00:15:00"


## Setup

All the manipulations and plots in this notebook can be created with standard libraries such as matplotlib, statsmodels etc. 

In [1]:
# Main data packages. 
import numpy as np
import pandas as pd

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name



## Import Data 

The data for this notebook was downloaded from the [meteoblue website](https://www.meteoblue.com/en/weather/archive/export/basel_switzerland_2661604) and consits of weather data for the city of Basel from 2008 till 2020. 

In [3]:

df_price = pd.read_csv("../../data_cleaned/by_source/ENERGY_PRICES.csv", delimiter=",")


In [7]:
df_price.shape

(96336, 12)

In [23]:
df_res_offshore = pd.read_csv("../../data_cleaned/by_source/RES_WIND_OFFSHORE.csv", delimiter=",")



In [24]:
df_res_offshore.shape

(96432, 13)

In [22]:

df_res_solar = pd.read_csv("../../data_cleaned/by_source/RES_WIND_SOLAR.csv", delimiter=",")

In [25]:
df_res_solar.shape

(96432, 13)

In [5]:
df_res_offshore.columns

Index(['date', 'year', 'month', 'day', 'dayofyear', 'hour', 'week',
       'dayofweek', 'off_wind_da', 'off_wind_act', 'period_start_utc',
       'period_end_utc', 'c_by_hour'],
      dtype='object')

In [27]:
from functools import reduce

#dfs = [df_price, df_res_offshore] ##, df_res_offshore, df_res_onshore, df_res_solar]  investigate on keys df_gen_forecast
dfs = [
    df_price,
    df_res_offshore[["period_start_utc","off_wind_da","off_wind_act"]],
    df_res_solar[["period_start_utc","solar_da","solar_act"]],
]

df_merged = reduce(
    lambda left, right: pd.merge(left, right, on="period_start_utc", how="left"),
    dfs
)

In [28]:
df_merged.shape

(96336, 16)

In [29]:
df_merged

,date,year,month,day,dayofyear,hour,week,dayofweek,price,period_start_utc,period_end_utc,c_by_hour,off_wind_da,off_wind_act,solar_da,solar_act
0,2015-01-04,2015,1,4,4,23,1,6,22.3400,2015-01-04 23:00:00,2015-01-05 00:00:00,1,192.1250,478.0200,0.0,0.1700
1,2015-01-05,2015,1,5,5,0,2,0,17.9300,2015-01-05 00:00:00,2015-01-05 01:00:00,1,192.3750,468.5525,0.0,0.2075
2,2015-01-05,2015,1,5,5,1,2,0,15.1700,2015-01-05 01:00:00,2015-01-05 02:00:00,1,195.0000,465.5550,0.0,0.1800
3,2015-01-05,2015,1,5,5,2,2,0,16.3800,2015-01-05 02:00:00,2015-01-05 03:00:00,1,193.6250,454.0175,0.0,0.2000
4,2015-01-05,2015,1,5,5,3,2,0,17.3800,2015-01-05 03:00:00,2015-01-05 04:00:00,1,187.6250,457.4750,0.0,0.1950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96331,2025-12-31,2025,12,31,365,18,1,2,95.9900,2025-12-31 18:00:00,2025-12-31 19:00:00,4,5430.0475,5418.4900,0.0,0.0675
96332,2025-12-31,2025,12,31,365,19,1,2,86.8025,2025-12-31 19:00:00,2025-12-31 20:00:00,4,5494.9900,5015.0525,0.0,0.0525
96333,2025-12-31,2025,12,31,365,20,1,2,79.7975,2025-12-31 20:00:00,2025-12-31 21:00:00,4,5359.1000,4390.0575,0.0,0.0575
96334,2025-12-31,2025,12,31,365,21,1,2,81.3900,2025-12-31 21:00:00,2025-12-31 22:00:00,4,5389.0800,3363.9350,0.0,0.0400


In [30]:
df_merged.to_csv("../../data_cleaned/merged/Merge_prices_offshore_wind_solar.csv", index=False)